In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision.models import resnet50, ResNet50_Weights

In [2]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)